In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# CHURN PREDICTION

PROBLEM STATEMENT
WHAT CUSTOMER SEGEMETNS ARE MORE LIKELY TO CHURN BALANCES IN THE NEXT QUARTER BY AT LEAST 50% CONSIDERING CURRENT QUARTER.

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pylab as py
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore')
sns.set(style = 'white')

In [ ]:
df = pd.read_csv("../input/banking-churn-prediction/Banking_churn_prediction.csv") 

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#Printing all the columns presenr in the data
df.columns

# "VARIABLE IDENTIFICATION"

In [ ]:
#Find out the data types of different variables
df.dtypes

In [ ]:
#identify list of variables with data type = int64
df.dtypes[df.dtypes=='int64']

#Summary:
1) customer_id is a unique assigned to customers. So, integers datatype is correctly specified.
2) branch_code again represents different branches, therefore it should be converted as "Category datatype.
3) Age and vintage are also numbers and hence we are okay wit them as integers.
4) Customer _networth_category is supposed to be an ordinal category, should be converted to "Category" datatype.
5) Churn 1 represents the churn and 0 represents not churn.However, there is no comparision between the two categories .This needs to be converted to each "Category" datatype.


In [ ]:
#Converting required variabes to "Category" datatype.
df['churn'] = df['churn'].astype('category')
df['branch_code']=df['branch_code'].astype('category')
df['customer_nw_category']=df['customer_nw_category'].astype('category')

In [ ]:
df.dtypes[df.dtypes=='int64']

In [ ]:
#FLOAT DATA TYPE
df.dtypes[df.dtypes=='float64']

#Summary
1) dependent is expected to be a whole number. Therefore we should first count the frequency of different numbers and than using feature engineering technique "Combining Sparse Category" should combine the lease happening whole numbers and than change it to category data type.
2) city varibale is also a unique code of a city represented by some integr number. Should be converted to "Category" type.
3) Rest of the variables are best represented by float64.


In [ ]:
df['dependents'].value_counts(normalize = True)

In [ ]:
fil = (df['dependents'] == 4) | (df['dependents'] == 5) | (df['dependents'] == 6) | (df['dependents'] == 7) | (df['dependents'] == 32) | (df['dependents'] == 50) | (df['dependents'] == 36) | (df['dependents'] == 52) | (df['dependents'] == 8) | (df['dependents'] == 9) | (df['dependents'] == 25)
df.loc[fil, 'dependents'] = 3
df['dependents'].value_counts(normalize = True)

In [ ]:
#Converint required variables to "Category" and "Integer" datatypes.
df['dependents'] = df['dependents'].astype('category')
df['city']=df['city'].astype('category')
df[['city','dependents']].dtypes

In [ ]:
df.dtypes

In [ ]:
#variables like 'GENDER','OCCUPATION'and 'LAST_TRANSACTION' are of object type. This means that PANDAS WAS NOT ABLE TO recognise 
#THE datatypes OF these three variables
df[['gender','occupation','last_transaction']].head(7)

1) gender and occupation variables belong to a categorical data types.
2)  last_transaction should be a datetime variable.

In [ ]:
df['gender']=df['gender'].astype('category')
df['occupation'] = df['occupation'].astype('category')
df[['gender','occupation']].dtypes

In [ ]:
#Creating an instance(data) of Datetimeindex class using "last_transaction"
df['last_transaction']=pd.DatetimeIndex(df['last_transaction'])

In [ ]:
#extracting new columns from "last_transaction"
date = pd.DatetimeIndex(df['last_transaction'])
#LAST DAY OF YEAR WHEN TRANSACTION WAS DONE
df['doy_ls_tran'] =date.dayofyear
#WEEK OF YEAR WHEN LAST TRANSACTION WAS DONE
df['woy_ls_tran']=date.weekofyear
#MONTH OF YEAR WHEN LAST TRANSACTION WAS DONE
df['moy_ls_tran']=date.month
#DAY OF WEEK WHEN LAST TRANSACTION WAS DONE
df['dow_ls_tran']=date.dayofweek


In [ ]:
df[['last_transaction','doy_ls_tran','woy_ls_tran','moy_ls_tran','dow_ls_tran']].head()

1) The first column is the complete data of the last transaction which was was done by the given customer.
2) The next column represent the day of year, week of year, month of year, day of week when last transaction was done. Breaking down the  date variable into these granual information will help us understand when the lasr transaction was done from different perspectives.Now that we have extracted the essentials from the 


In [ ]:
df.dtypes

# Here we complete our first step of "Exploratory Data Analytics" i.e. "Variable Identification"

# UNIVARIATE ANALYSIS

"CONTINUOUS VARIABLES"

In [ ]:
df.select_dtypes(include=['int64','float64','Int64']).dtypes

In [ ]:
# segregating variables into groups
customer_details = ['customer_id','vintage', 'age']
current_month = ['current_balance','current_month_credit','current_month_debit','current_month_balance']
previous_month = ['previous_month_end_balance','previous_month_credit','previous_month_debit','previous_month_balance']
previous_quarters = ['average_monthly_balance_prevQ','average_monthly_balance_prevQ2']
transaction_date = ['doy_ls_tran','woy_ls_tran','moy_ls_tran','dow_ls_tran']

In [ ]:
# custom function for easy and efficient analysis of numerical univariate
def UVA_numeric(df, var_group):
  ''' 
  Univariate_Analysis_numeric
  takes a group of variables (INTEGER and FLOAT) and plot/print all the descriptives and properties along with KDE.

  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,3), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    mini = df[i].min()
    maxi = df[i].max()
    ran = df[i].max()-df[i].min()
    mean = df[i].mean()
    median = df[i].median()
    st_dev = df[i].std()
    skew = df[i].skew()
    kurt = df[i].kurtosis()

    # calculating points of standard deviation
    points = mean-st_dev, mean+st_dev

    #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.kdeplot(df[i], shade=True)
    sns.lineplot(points, [0,0], color = 'black', label = "std_dev")
    sns.scatterplot([mini,maxi], [0,0], color = 'orange', label = "min/max")
    sns.scatterplot([mean], [0], color = 'red', label = "mean")
    sns.scatterplot([median], [0], color = 'blue', label = "median")
    plt.xlabel('{}'.format(i), fontsize = 20)
    plt.ylabel('density')
    plt.title('std_dev = {}; kurtosis = {};\nskew = {}; range = {}\nmean = {}; median = {}'.format((round(points[0],2),round(points[1],2)),
                                                                                                   round(kurt,2),

                                                                                                   round(skew,2),
                                                                                                   (round(mini,2),round(maxi,2),round(ran,2)),
                                                                                                   round(mean,2),
                                                                                                   round(median,2)))


In [ ]:
# Univariate Analysis of "Customer Details"
UVA_numeric(df, customer_details)

# Summary of Customer_Information:

1) customer_id:
#variable is unique for every customer, Hence uniform distribution.
#This variable does not contribute any information can be eliminated from data

2) age:
#Median Age = 46
#Most customers age between 30 to 66
#skewness +0.33 : customer age is negligibly biased towards younger age
#kurtosis = -0.17; very less likely to have extreme/outlier values.

3) vintage:
#Most customers joined between 2100 and 2650 days from the day of data extraction.
#skewness -1.42 : this is left skewed, vintage variable is significantly biased towards longer association of customers.
#Kurtosis = 2.93: Extreme values and Outliers are very likely to be present in vintage.


In [ ]:
#droping "customer_id" from the dataframe
df.drop(['customer_id'], axis = 1, inplace  = True) 

In [ ]:
#Univariate analysis of customer details
UVA_numeric(df, current_month)

#Summary
1) Considering the kurtosis and skewness value  for all 4 of these plots. Outliers/Extreme values are present.
2) Need to Remove Outliers to visulaise these plots**

In [ ]:
#Univariate analysis of previous month
UVA_numeric(df,previous_month)

#Summary of previous_month
1) This looks very similar to current_month. Most of the customers perform low amount transactions. Therefore we need to find out whether there exists a relationship beetween current month and previous month balances. This will done in Bivariate analysis.


In [ ]:
# Univariate analysis of transaction date
UVA_numeric(df,transaction_date)

#Summary


#Day_of_Year:
1) most of the last transactions were made in the last 60 days of the extraction of data.There are transactions which were made more than an year ago.
#Week_of_year and Month_of_year:
1)These variable validate the findings from the day_of_year.
#date of week figure shows no transaction on sunday as we all know the banks are closed on sunday. We can see high number of transaction on Monday.


# Univariate analysis Categorical variables

In [ ]:
def UVA_category(df, var_group):

  '''
  Univariate_Analysis_categorical
  takes a group of variables (category) and plot/print all the value_counts and barplot.
  '''
  # setting figure_size
  size = len(var_group)
  plt.figure(figsize = (7*size,5), dpi = 100)

  # for every variable
  for j,i in enumerate(var_group):
    norm_count = df[i].value_counts(normalize = True)
    n_uni = df[i].nunique()

  #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.barplot(norm_count, norm_count.index , order = norm_count.index)
    plt.xlabel('fraction/percent', fontsize = 20)
    plt.ylabel('{}'.format(i), fontsize = 20)
    plt.title('n_uniques = {} \n value counts \n {};'.format(n_uni,norm_count))

In [ ]:
UVA_category(df, ['occupation', 'gender', 'customer_nw_category','dependents'])    

#Summary

#Occupation
60% of people are self_employed.
There are 1% Company Accounts. Might explain Outliers Extreme values in credit/debit.

#Gender:
Males accounts are 1.5 times in number than Female Accounts i.e. we have 60% males with 40% females
#customer_nw_category:
Half of all the accounts belong to the 3rd net worth category.
Less than 15% belong to the highest net worth category.

#dependents : 
Majority of the population has 0 dependents. This means they have no one to take care off on their expenses.


In [ ]:
UVA_category(df, ['city', 'branch_code'])

In [ ]:
#CHURN

UVA_category(df, ['churn'])

#Number of people who churned are 1/4 times of the people who did not churn in the given data.

# ANALYSIS of  MISSING VALUES

In [ ]:

# finding number of missing values in every variable
df.isnull().sum()

#Things to investigate further down:**
#Gender:
1)do missing values have any relation with churn?

#Dependents:
1) Missing values might be similar to zero dependents
2) do missing values have any relation with churn?

#Occupation:
1)Do missing values have similar behaviour to any other occupation
2)do they have some relation with churn?

#city:
1)the respective cities can be found using branch_code

#last_transaction:
1)checking their previous month and current month and previous_quarter activity might give insight on their last transaction.
2)Due to missing values in last_transactions we are seeing missing values in "doy_ls_tran","woy_ls_tran","moy_ls_tran","dow_ls_tran"


# "OUTLIER ANALYSIS

In [ ]:
# Calculating Outliers
def UVA_outlier(df, var_group, include_outlier = True):
  '''
  Univariate_Analysis_outlier:
  takes a group of variables (INTEGER and FLOAT) and plot/print boplot and descriptives\n
  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it \n\n

  data : dataframe from which to plot from\n
  var_group : {list} type Group of Continuous variables\n
  include_outlier : {bool} whether to include outliers or not, default = True\n
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,4), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    quant25 = df[i].quantile(0.25)
    quant75 = df[i].quantile(0.75)
    IQR = quant75 - quant25
    med = df[i].median()
    whis_low = med-(1.5*IQR)
    whis_high = med+(1.5*IQR)

    # Calculating Number of Outliers
    outlier_high = len(df[i][df[i]>whis_high])
    outlier_low = len(df[i][df[i]<whis_low])

    if include_outlier == True:
      print(include_outlier)
      #Plotting the variable with every information
      plt.subplot(1,size,j+1)
      sns.boxplot(df[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('With Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2),
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))
      
    else:
      # replacing outliers with max/min whisker
      data = df[var_group][:]
      data[i][data[i]>whis_high] = whis_high+1
      data[i][data[i]<whis_low] = whis_low-1
      
      # plotting without outliers
      plt.subplot(1,size,j+1)
      sns.boxplot(data[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('Without Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2)
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))  
  


In [ ]:
UVA_outlier(df, current_month)

In [ ]:
UVA_outlier(df, previous_month)

Summary
1)If we look at corresponding plots in the outputs above, there seems to be a strong relation between the corresponding plots of previous_month and current_month variables.

2)Outliers are significant in number and very similar in number between corresponding plots. Which indicates some inherent undiscovered behviour of Outliers


In [ ]:
UVA_outlier(df,previous_quarters)

#Outliers in previous two quarters are very similar but significantly large in number.
 
#Investigation directions from Univariate Analysis
#customer_id variable can be dropped.
#Is there there any common trait/relation between the customers who are performing high transaction credit/debits?
#customer_nw_category might explain that.
#Occupation = Company might explain them
#popular cities might explain this
#4Customers whose last transaction was 6 months ago, did all of them churn? 
#Possibility that cities and branch code with very few accounts may lead to churning.

In [ ]:
customer_details = ['vintage', 'age',]

In [ ]:
UVA_outlier(df,customer_details)

#Vintage has some outliers at the lower side.
#Age seems to have no outliers.

# BIVARIATE ANALYSIS

NUMERICAL NUMERICAL

In [ ]:
numerical = df.select_dtypes(include=['int64','float64','Int64'])
numerical.dtypes


In [ ]:
### CORRELATION MATRIX
# calculating correlation
correlation = numerical.dropna().corr()
correlation


In [ ]:
### HEATMAP
# plotting heatmap usill all methods for all numerical variables
plt.figure(figsize=(36,6), dpi=140)
for j,i in enumerate(['pearson','kendall','spearman']):
  plt.subplot(1,3,j+1)
  correlation = numerical.dropna().corr(method=i)
  sns.heatmap(correlation, linewidth = 2)
  plt.title(i, fontsize=18)

#Summary
1) Kendall and Spearman correlation seem to have very similar pattern between them, except the slight variation in magnitude of correlation.
2) Too many variables with insignificant correlation.
3) Major correlation lies between the transaction variables and balance variables.  
4) extracting transaction information of current and previous months

In [ ]:
     var = []
var.extend(previous_month)
var.extend(current_month)
var.extend(previous_quarters)  

# plotting heatmap usill all methods for all transaction variables
plt.figure(figsize=(36,6), dpi=140)
for j,i in enumerate(['pearson','kendall','spearman']):
  plt.subplot(1,3,j+1)
  correlation = numerical[var].dropna().corr(method=i)
  sns.heatmap(correlation, linewidth = 2)
  plt.title(i, fontsize=18)

#Transaction variables like credit/debit have a strong correlation among themselves.
#Balance variables have strong correlation among themselves.
#Transaction variables like credit/debit have insignificant or no correlation with the Balance variables.

CONTINUOUS CATEGORICAL VARIABLES

# List of Hypothesis and investigation to perform under this combination.
1) Are vintage customers less likely to churn?
2) Are customers with higher average balance less likely to churn?
3) Are customers dropping monthly balance highly likely to churn?
4) Do age affect the decisions of churning?

In [ ]:
def TwoSampZ(X1, X2, sigma1, sigma2, N1, N2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sampled Z-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import norm
  ovr_sigma = sqrt(sigma1**2/N1 + sigma2**2/N2)
  z = (X1 - X2)/ovr_sigma
  pval = 2*(1 - norm.cdf(abs(z)))
  return pval


In [ ]:
def TwoSampT(X1, X2, sd1, sd2, n1, n2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sample T-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import t as t_dist
  ovr_sd = sqrt(sd1**2/n1 + sd2**2/n2)
  t = (X1 - X2)/ovr_sd
  df = n1+n2-2
  pval = 2*(1 - t_dist.cdf(abs(t),df))
  return pval

In [ ]:
def Bivariate_cont_cat(df, cont, cat, category):
  #creating 2 samples
  x1 = df[cont][df[cat]==category][:]
  x2 = df[cont][~(df[cat]==category)][:]
  
  #calculating descriptives
  n1, n2 = x1.shape[0], x2.shape[0]
  m1, m2 = x1.mean(), x2.mean()
  std1, std2 = x1.std(), x2.mean()
  
  #calculating p-values
  t_p_val = TwoSampT(m1, m2, std1, std2, n1, n2)
  z_p_val = TwoSampZ(m1, m2, std1, std2, n1, n2)

  #table
  table = pd.pivot_table(data=df, values=cont, columns=cat, aggfunc = np.mean)

  #plotting
  plt.figure(figsize = (15,6), dpi=140)
  
  #barplot
  plt.subplot(1,2,1)
  sns.barplot([str(category),'not {}'.format(category)], [m1, m2])
  plt.ylabel('mean {}'.format(cont))
  plt.xlabel(cat)
  plt.title('t-test p-value = {} \n z-test p-value = {}\n {}'.format(t_p_val,
                                                                z_p_val,
                                                                table))

  # boxplot
  plt.subplot(1,2,2)
  sns.boxplot(x=cat, y=cont, data=df)
  plt.title('categorical boxplot')


In [ ]:
### 1. Are vintage customers less likely to churn?
Bivariate_cont_cat(df, 'vintage', 'churn', 1)  

#Inferences

#Vintage customers churned more, but results are not significantly different
#Boxplot shows very similar distribution with outliers on the lower end.
#Result
#Because of higher Pvalue we can safely reject the alternative hypothesis that vintage customers are less likely to churn.


In [ ]:
### 2. Are customers with higher average balance less likely to churn?
Bivariate_cont_cat(df, 'average_monthly_balance_prevQ', 'churn', 1)

#Customers who churned have significantly higher balance during immediate preceeding quarter.
#Smaller Pvalue suggests that we reject null hypothesis.

In [ ]:
#### 2. Are customers with higher average balance in previour quarter 2 less likely to churn?
Bivariate_cont_cat(df, 'average_monthly_balance_prevQ2', 'churn', 1)

#We can see that people who churned actually had significantly highe balance during their previous two quarters.

In [ ]:
#Are customers with higher customer balance less likely to churn
Bivariate_cont_cat(df, 'current_month_balance', 'churn', 1)

#Due to high p value we can say that the current balance value do not significantly differentiate between churning and non churning customers.

#Inferences

#Customers who churned had significantly high balance throughout the previous two quarters and previous month. But their average balance reduced significantly in the current month.


In [ ]:
### 3. Are customers dropping monthly balance highly likely to churn?

# Extracting drop of balance in previous and current month
difference = df[['churn','previous_month_balance','current_month_balance']][:]
difference['bal_diff'] = difference['current_month_balance']-difference['previous_month_balance']

In [ ]:
Bivariate_cont_cat(difference, 'bal_diff', 'churn', 1)

#Inference
#Customers who churned had a very high drop in their balance which is signified by the negative value in this bar plot.


In [ ]:
Bivariate_cont_cat(df, 'age', 'churn', 1)

#We can see that their is a significant diffetence in the age of customets who chur and who do not churn.

"CATEGORICAL CATEGORICAL"

#### List of Hypothesis to check under this combination
#Are females less likely to churn than males?
#Are young customers more likely to churn?
#Are customers in the lower income bracket more likely to churn?
#Are customers with dependent(s) less likely to churn?
#Customers with an average family size less than 4 are more likely to churn?
#Possibility that cities and branch code with very few accounts may lead to churning.

In [ ]:
#defining function fot categorical-categorical bivariate analysis
def BVA_categorical_plot(data, tar, cat):
  '''
  take data and two categorical variables,
  calculates the chi2 significance between the two variables 
  and prints the result with countplot & CrossTab
  '''
  #isolating the variables
  data = df[[cat,tar]][:]

  #forming a crosstab
  table = pd.crosstab(df[tar],df[cat],)
  f_obs = np.array([table.iloc[0][:].values,
                    table.iloc[1][:].values])

  #performing chi2 test
  from scipy.stats import chi2_contingency
  chi, p, dof, expected = chi2_contingency(f_obs)
  
  #checking whether results are significant
  if p<0.05:
    sig = True
  else:
    sig = False

  #plotting grouped plot
  sns.countplot(x=cat, hue=tar, data=data)
  plt.title("p-value = {}\n difference significant? = {}\n".format(round(p,8),sig))

  #plotting percent stacked bar plot
  #sns.catplot(ax, kind='stacked')
  ax1 = df.groupby(cat)[tar].value_counts(normalize=True).unstack()
  ax1.plot(kind='bar', stacked='True',title=str(ax1))
  int_level = df[cat].value_counts()

In [ ]:
#Are females less likely to churn than males?  
BVA_categorical_plot(df, 'churn', 'gender')

#Result:
#the difference between the males and females customer churning is significant and more males customers churn.

In [ ]:
#Dependents  
BVA_categorical_plot(df, 'churn', 'dependents')

# There seems to be significant between the churning rate for different dependents categories. Persons with no dependents are more likely to churn.

In [ ]:
#Occupation  
BVA_categorical_plot(df, 'churn', 'occupation')

#There seems to be statistically significant difference between the churning rate in different occupation categories. Also, self employed has highest probability to churn.

In [ ]:
#CUSTOMER FROM LOW INCOME BRACKET MORE LIKELY TO CHURN
BVA_categorical_plot(df, 'churn', 'customer_nw_category')

#Result:
#Different income brackets have significant effect on the churn rate.


In [ ]:
# segregating customers into segments
#Are young customers more likely to churn

df['age_group'] = " "
df['age_group'] = 'categorical'
df['age_group'][df['age']>=80] = 'very old'
df['age_group'][(df['age']<80) & (df['age']>=60)] = 'senior citizen'
df['age_group'][(df['age']<60) & (df['age']>=18)] = 'adult'
df['age_group'][df['age']<18] = 'young'

In [ ]:
BVA_categorical_plot(df,'churn','age_group')

#Age group has significant effect on churning rate. However we can see major churning customers lies in adult and senior age group

In [ ]:
#Possibilities that cities and branch code with very few accounts may lead to churning
# getting city codes which have less than 280 (1%) of accounts
tmp = df['city'].value_counts()[:]
cities = tmp[tmp<280].index
df['city_cat'] = ' '
df['city_cat'] = None
df['city_cat'][df['city'].isin(cities[:])] = 'low accounts'
df['city_cat'][~df['city'].isin(cities[:])] = 'high accounts'

In [ ]:
BVA_categorical_plot(df, 'churn', 'city_cat')

#cities having less than 1 percent of the total have significantly different churn rates as compared to the cities with more accounts.

In [ ]:
#Branch Code
# getting branch codes with more than 0.5% of total accounts
tmp = df['branch_code'].value_counts()[:]
branch = tmp[tmp<140].index


In [ ]:
# making two segments
df['branch_cat'] = ' ' 
df['branch_cat'][df['branch_code'].isin(branch[:])] = 'low accounts'
df['branch_cat'][~df['branch_code'].isin(branch[:])] = 'high accounts'

In [ ]:
BVA_categorical_plot(df, 'churn', 'branch_cat')

#There is no significant difference between the high account count branches and low account count branches.

# PREPROCESSING

Missing Values
#Before we go on to build the model, we must look for missing values within

#the dataset as treating the missing values is a necessary step before we

#fit a logistic regression model on the dataset.

In [ ]:
pd.isnull(df).sum()

#The result of this function shows that there are quite a few missing values

#in columns gender, dependents, city, days since last transaction and Percentage change in credits.

#Let us go through each of them 1 by 1 to find the appropriate missing value imputation strategy for each of them.

In [ ]:
df['gender'].value_counts()

#So there is a good mix of males and females and arguably missing values

#cannot be filled with any one of them. We could create a seperate category

#by assigning the value -1 for all missing values in this column.

#Before that, first we will convert the gender into 0/1 and then replace missing values with -1

In [ ]:
dict_gender = {'Male': 1, 'Female':0}
df.replace({'gender': dict_gender}, inplace = True)
df['gender'] = df['gender'].fillna(-1)

In [ ]:
df['dependents'].value_counts()

In [ ]:
df['dependents'] = df['dependents'].fillna(0)

In [ ]:
df['occupation'].value_counts()

In [ ]:
df['occupation'] = df['occupation'].fillna('self_employed')

In [ ]:
df['city'].value_counts()

In [ ]:
df['city'].mode()

In [ ]:
#Similarly City can also be imputed with most common category 1020
df['city'] = df['city'].fillna(1020)


#Days since Last Transactions
#A fair assumption can be made on this column as this is number of days 
#since last transaction in 1 year, we can substitute missing values with a 
#value greater than 1 year say 999

In [ ]:
df['last_transaction'] = df['last_transaction'].fillna(999)

#Now, before applying linear model such as logistic regression, we need to
#scale the data and keep all features as numeric strictly.


In [ ]:
df.dtypes

In [ ]:
df['gender']=df['gender'].astype('category')

In [ ]:
df1 = df[['vintage', 'age', 'gender', 'dependents', 'occupation',
       'customer_nw_category','current_balance',
       'previous_month_end_balance', 'average_monthly_balance_prevQ',
       'average_monthly_balance_prevQ2', 'current_month_credit',
       'previous_month_credit', 'current_month_debit', 'previous_month_debit',
       'current_month_balance', 'previous_month_balance', 'churn','age_group','city_cat']]

In [ ]:
### Preprocessing
#Now, before applying linear model such as logistic regression, we need to
#scale the data and keep all features as numeric strictly.

#DUMMIES WITH MULTIPLE CATEGORIES
# Convert occupation to one hot encoded features
df1 = pd.concat([df1,pd.get_dummies(df1['occupation'],prefix = str('occupation'),prefix_sep='_')],axis = 1)
df1= pd.concat([df1,pd.get_dummies(df1['gender'],prefix = str('gender'),prefix_sep='_')],axis = 1)
df1= pd.concat([df1,pd.get_dummies(df1['customer_nw_category'],prefix = str('customer_nw_category'),prefix_sep='_')],axis = 1)
df1= pd.concat([df1,pd.get_dummies(df1['dependents'],prefix = str('dependents'),prefix_sep='_')],axis = 1)
df1= pd.concat([df1,pd.get_dummies(df1['age_group'],prefix = str('age_group'),prefix_sep='_')],axis = 1)
df1= pd.concat([df1,pd.get_dummies(df1['city_cat'],prefix = str('city_cat'),prefix_sep='_')],axis = 1)

#Scaling Numerical Features for Logistic Regression

#Now, we remember that there are a lot of outliers in the dataset especially

#when it comes to previous and current balance features. Also, the

#distributions are skewed for these features if you recall from the EDA.

#We will take 2 steps to deal with that here:

#Log Transformation

#Standard Scaler

#Standard scaling is anyways a necessity when it comes to linear models

#and we have done that here after doing log transformation on all balance

#features.

In [ ]:
num_cols = [ 'current_balance','vintage',
            'previous_month_end_balance', 'average_monthly_balance_prevQ2', 'average_monthly_balance_prevQ',
            'current_month_credit','previous_month_credit', 'current_month_debit', 
            'previous_month_debit','current_month_balance', 'previous_month_balance']

In [ ]:
for i in num_cols:
    df1[i] = np.log(df1[i] + 17000)

In [ ]:
std = StandardScaler()
scaled = std.fit_transform(df1[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)
df_df_og = df1.copy()
df1 = df1.drop(columns = num_cols,axis = 1)
df1= df1.merge(scaled,left_index=True,right_index=True,how = "left")

In [ ]:
y_all = df1.churn

In [ ]:
df1 = df1.drop(['churn','occupation','gender','dependents','customer_nw_category','city_cat','age_group'],axis = 1)

In [ ]:
df1.columns

In [ ]:
baseline_cols = ['current_month_debit', 'previous_month_debit','current_balance','previous_month_end_balance','vintage'
                 ,'occupation_retired', 'occupation_salaried','occupation_self_employed', 'occupation_student','age','gender_-1.0', 'gender_0.0', 'gender_1.0', 'customer_nw_category_1',
       'customer_nw_category_2', 'customer_nw_category_3', 'dependents_0.0',
       'dependents_1.0', 'dependents_2.0', 'dependents_3.0', 'age_group_adult',
       'age_group_senior citizen', 'age_group_very old', 'age_group_young','city_cat_high accounts', 'city_cat_low accounts']

In [ ]:
df_baseline = df1[baseline_cols]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(df_baseline,y_all,test_size=1/3, random_state=11, stratify = y_all)
model = LogisticRegression()
model.fit(xtrain,ytrain)
pred = model.predict_proba(xtest)[:,1]

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(ytest,pred) 
auc = roc_auc_score(ytest, pred) 
plt.figure(figsize=(12,8)) 
plt.plot(fpr,tpr,label="Validation AUC-ROC="+str(auc)) 
x = np.linspace(0, 1, 1000)
plt.plot(x, x, linestyle='-')
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.legend(loc=4) 
plt.show()

In [ ]:
# Confusion Matrix
pred_val = model.predict(xtest)

label_preds = pred_val

cm = confusion_matrix(ytest,label_preds)


def plot_confusion_matrix(cm, normalized=True, cmap='bone'):
    plt.figure(figsize=[7, 6])
    norm_cm = cm
    if normalized:
        norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=['Predicted: No','Predicted: Yes'], yticklabels=['Actual: No','Actual: Yes'], cmap=cmap)

plot_confusion_matrix(cm, ['No', 'Yes'])

In [ ]:
recall_score(ytest,pred_val)

In [ ]:
### Cross validation


#Cross Validation is one of the most important concepts in any type of data modelling. It simply says, try to leave a sample on which you do not train the model and test the model on this sample before finalizing the model.
#We divide the entire population into k equal samples. Now we train models on k-1 samples and validate on 1 sample. Then, at the second iteration we train the model with a different sample held as validation. 
#In k iterations, we have basically built model on each sample and held each of them as validation. This is a way to reduce the selection bias and reduce the variance in prediction power.
#Since it builds several models on different subsets of the dataset, we can be more sure of our model performance if we use CV for testing our models.


def cv_score(ml_model, rstate = 12, thres = 0.5, cols = df1.columns):
    i = 1
    cv_scores = []
    df2 = df1.copy()
    df2 = df1[cols]
    
    # 5 Fold cross validation stratified on the basis of target
    kf = StratifiedKFold(n_splits=5,random_state=rstate,shuffle=True)
    for df_index,test_index in kf.split(df2,y_all):
        print('\n{} of kfold {}'.format(i,kf.n_splits))
        xtr,xvl = df2.loc[df_index],df2.loc[test_index]
        ytr,yvl = y_all.loc[df_index],y_all.loc[test_index]
            
        # Define model for fitting on the training set for each fold
        model = ml_model
        model.fit(xtr, ytr)
        pred_probs = model.predict_proba(xvl)
        pp = []
         
        # Use threshold to define the classes based on probability values
        for j in pred_probs[:,1]:
            if j>thres:
                pp.append(1)
            else:
                pp.append(0)
         
        # Calculate scores for each fold and print
        pred_val = pp
        roc_score = roc_auc_score(yvl,pred_probs[:,1])
        recall = recall_score(yvl,pred_val)
        precision = precision_score(yvl,pred_val)
        sufix = ""
        msg = ""
        msg += "ROC AUC Score: {}, Recall Score: {:.4f}, Precision Score: {:.4f} ".format(roc_score, recall,precision)
        print("{}".format(msg))
         
         # Save scores
        cv_scores.append(roc_score)
        i+=1
    return cv_scores


In [ ]:
baseline_scores = cv_score(LogisticRegression(), cols = baseline_cols)

In [ ]:
#There is some improvement in both ROC AUC Scores and Precision/Recall Scores. Now we can try backward selection to select the best subset of features which give the best score. 
all_feat_scores = cv_score(LogisticRegression())

In [ ]:
#Reverse Feature Elimination and Backward Selection
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

In [ ]:
#just copies the data frame to a new place
df2 = df1
df2.dtypes
#just drop all the object categories
# Create the RFE object and rank each feature
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=1, step=1)
rfe.fit(df1, y_all)
ranking_df = pd.DataFrame()
ranking_df['Feature_name'] = df1.columns
ranking_df['Rank'] = rfe.ranking_

ranked = ranking_df.sort_values(by=['Rank'])
ranked

#The balance features are proving to be very important as can be seen from the table. The RFE function can also be used to select features. Lets select the top 10 features from this table and check score.

rfe_top_10_scores = cv_score(LogisticRegression(), cols = ranked['Feature_name'][:10].values)

#Wow, the top 10 features obtained using the reverse feature selection are giving a much better score than any of our earlier attempts. This is the power of feature selection and it especially works well in case of linear models as tree based models are in itself to some extent capable of doing feature selection.
#The recall score here is quite low. We should play around with the threshold to get a better recall score. AUC ROC depends on the predicted probabilities and is not impacted by the threshold. Let us try 0.2 as threshold which is close to the overall churn rate
cv_score(LogisticRegression(), cols = ranked['Feature_name'][:10].values, thres=0.14)

#We observe that there is continuous improvement in the Recall Score. However, clearly precision score is going down. On the basis of business requirement the bank can take a call on deciding the threshold. Without knowing the metrics relevant to the business, our best course of action is to optimize for AUC ROC Score so as to find the best probabilites here.
#Comparison of Different model fold wise
#Let us visualise the cross validation scores for each fold for the following 3 models and observe differences:


In [ ]:
#Baseline Model
#Model based on all features
#Model based on top 10 features obtained from RFE
results_df = pd.DataFrame({'baseline':baseline_scores, 'all_feats': all_feat_scores, 'rfe_top_10': rfe_top_10_scores})

results_df.plot(y=["baseline", "all_feats", "rfe_top_10"], kind="bar")
#Here, we can see that the model based on RFE is giving the best result for each fold and students are encouraged to try more feature selection techniques and fine tune to get the best results.